# Scipt for using VK API to retrieve the data

In [115]:
# users ids were previously generated:
# import random
# vk_users_n=710000000
# id_list=[random.randrange(1, vk_users_n+1, 1) for i in range(30000)]

In [1]:
# open file  файл with ids
with open("../data/id_generated.txt", mode="r") as file:
    
    f = file.read()
    id_list = [int(i) for i in f.split(",")]
    print (len(id_list))


30000


In [35]:
# VK Authorization
# note that there is a day limit for requests with vk api. 5000 requests per day.
import vk
import json
import requests

access_token = "#"
ac2 = "##"
ac_n ="###""
ac_m="####"

v='5.92'

In [117]:
#====  Get posts by id  =======

day_limit=5000
res_folder='../results_from_rnd_walls2/'

import numpy as np
import pandas as pd
import time
from datetime import datetime

post_df=pd.DataFrame(columns=['user_id', 'post_id', 'date', 'text', 'link_title', 'link_description', 'repost_text', 'views', 'likes', 'comments', 'reposts'])

# part_number=int(len(id_list)/day_limit) # calculate the number of parts to devide, because of day limit 
# for i in range(0,part_number+1):
#     print ('part_number '+str(i))    
#     step=i*day_limit
#     id_list_=id_list[step:step+day_limit] # partial id list - because of day_limit (5000) per day

# progress bar tqdm
from tqdm.notebook import tqdm
pbar = tqdm(id_list_)
pbar.set_description("Processing id_s")

pbar_concat = tqdm()
pbar_concat.set_description("Last added for None")

requests_tick = datetime.now()
td_hold = (1.05/3)

for i, id_ in enumerate(pbar):
        
    offset=0
    
    while True:
        
        td = (datetime.now() - requests_tick).total_seconds()
        if td < td_hold:
            time.sleep(td_hold - td)
        
        requests_tick = datetime.now()        
        try:
            q = requests.get('https://api.vk.com/method/wall.get', params={
                        'access_token':access_token,  # or ac2, ac_n, ac_m. you can combine multiple tokens to speed up the retrieving process
                        'v':v,
                        'owner_id':id_,
                        'count':100,
                        'offset':offset,
                        'filter':'owner'
                        }).json()['response']
        except Exception:
            break
            
        offset=offset+100        
        
        
        if len(q['items']) == 0:
            break
        
        for post in q['items']:
                post_list=[]
                post_list.append(id_)
                post_list.append(post.get('id', -9999))
                full_date=datetime.fromtimestamp(post['date'])
                post_list.append(datetime.date(full_date))
                text_=post['text'].replace("\n"," ")
                text=text_.replace("|"," ")
                post_list.append(text)


                get_record=False 
                if 'attachments' in post:
                    for att in post['attachments']: # attachments type - may be foto, doc, video, link.We only need a link, but there can be several types in attachments, so we check if the link is found 
                        if (att['type']=='link'):
                            post_list.append(att.get('link', {'title':'',}).get('title', ''))
                            post_list.append(att.get('link', {'description':'',}).get('description', ''))
                            get_record=True
                if not get_record:
                    post_list.append("")
                    post_list.append("")
                
                
                # If the repost is with a comment, then there are several elements in the history, with several texts, we take only the user's text (he is the first one)
#                 ind=0                                 
                if 'copy_history' in post:
                    for att in post['copy_history']:
                        if att.get('text', '')!=u"Запись удалена":
                            post_list.append(att.get('text', '').replace("\n"," "))
    #                         ind=ind+1
                            break
                else:
                    post_list.append('')

                #==== We check if all the elements of the list are empty (except for id_s, date). Can be empty, for example, if only a photo is added
                k=0
                for i in range(3, len(post_list)-3):
                    if len(str(post_list[i]))>0:
                        k=k+1
                
                if k>0:
                #======Only if there is a text/repost of a post or story, we collect information about views/likes/reposts/ ================================================================

                    post_list.append(post.get('views', {'count':'',}).get('count', ''))
                    post_list.append(post.get('likes', {'count':'',}).get('count', ''))
                    post_list.append(post.get('comments', {'count':'',}).get('count', ''))
                    post_list.append(post.get('reposts', {'count':'',}).get('count', ''))


                    post_df = pd.concat([post_df, pd.DataFrame(pd.Series(post_list, index=post_df.columns)).T], axis=0, ignore_index=True)
                    pbar_concat.set_description("Last added for  %s" % id_)
                    pbar_concat.update()

#save here the dataframe

  0%|          | 0/80000 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [1]:
# save all parts in the folder
# You need to look at how many ids from the id_list did not appear in the final table of results (because vk api often skips ids). We find those that were skiped, and retry
